<a href="https://colab.research.google.com/github/Charles8745/gans-in-action/blob/master/gans_in_action_CH4_DCGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>